# 问题

- 你想直接读取二进制数据到一个可变缓冲区中，而不需要做任何的中间复制操作。或者你想原地修改数据并将它写回到一个文件中去。

## 解决方案

- 为了读取数据到一个可变数组中，使用文件对象的readinto() 方法。比如：

In [2]:
import os

def read_info_buffer(filename):
    # 创建一个缓冲区
    buf = bytearray(os.path.getsize(filename))
    with open(filename, 'rb') as f:
        f.readinto(buf)
        
    return buf

In [7]:
buf = read_info_buffer('bin.bin')
print(buf)
print(buf.decode())

buf[0:5] = b'hello'

print(buf.decode())

bytearray(b'\x01\x00\x00\x00\x02\x00\x00\x00\x03\x00\x00\x00\x04\x00\x00\x00')
            
hello         


## 讨论

- 文件对象的readinto() 方法能被用来为预先分配内存的数组填充数据，甚至包括由array 模块或numpy 库创建的数组。和普通read() 方法不同的是，readinto() 填充已存在的缓冲区而不是为新对象重新分配内存再返回它们。因此，你可以使用它来避免大量的内存分配操作。比如，如果你读取一个由相同大小的记录组成的二进制文件时，你可以像下面这样写：

In [11]:
SIZE = 4

buf = bytearray(SIZE)
with open('text.txt', 'rb') as f:
    while True:
        n = f.readinto(buf)
        if n < SIZE:
            break
        print(buf)

bytearray(b'\xe8\xbf\x99\xe6')
bytearray(b'\x98\xaf\xe4\xb8')
bytearray(b'\x80\xe4\xb8\xaa')
bytearray(b'\xe6\x96\x87\xe6')
bytearray(b'\x9c\xac\xe6\x96')
bytearray(b'\x87\xe4\xbb\xb6')
bytearray(b'\n\xe8\xbf\x99')
bytearray(b'\xe6\x98\xaf\xe6')
bytearray(b'\x96\x87\xe6\x9c')
bytearray(b'\xac\xe6\x96\x87')
bytearray(b'\xe4\xbb\xb6\xe7')
bytearray(b'\x9a\x84\xe7\xac')
bytearray(b'\xac\xe4\xba\x8c')


- 另外有一个有趣特性就是memoryview ，它可以通过零复制的方式对已存在的缓冲区执行切片操作，甚至还能修改它的内容。比如：